# 📊 Dashboard Interactivo con Streamlit

En este notebook aprenderemos a crear dashboards interactivos usando Streamlit.

**Nota**: Streamlit se ejecuta como aplicación web, no directamente en Jupyter. Este notebook sirve para desarrollar y probar el código antes de crear el archivo `app.py`.


In [1]:
%pip install -q streamlit pandas plotly altair openpyxl

import pandas as pd
import numpy as np
import plotly.express as px
import plotly.graph_objects as go
from datetime import datetime, timedelta
import warnings
warnings.filterwarnings('ignore')

print("✅ Librerías cargadas correctamente")


Note: you may need to restart the kernel to use updated packages.
✅ Librerías cargadas correctamente


## 2. Crear dataset de ejemplo

Generamos un dataset sintético de ventas para el dashboard.


In [2]:
# Crear dataset sintético de ventas
np.random.seed(42)

n_rows = 1000
categorias = ['Electrónica', 'Ropa', 'Hogar', 'Deportes', 'Libros']
regiones = ['Norte', 'Sur', 'Este', 'Oeste', 'Centro']
vendedores = ['Ana García', 'Carlos López', 'María Rodríguez', 'Juan Martínez', 'Laura Sánchez']

# Generar fechas aleatorias en 2024
start_date = datetime(2024, 1, 1)
dates = [start_date + timedelta(days=np.random.randint(0, 365)) for _ in range(n_rows)]

df = pd.DataFrame({
    'fecha': dates,
    'categoria': np.random.choice(categorias, n_rows),
    'region': np.random.choice(regiones, n_rows),
    'vendedor': np.random.choice(vendedores, n_rows),
    'cantidad': np.random.randint(1, 20, n_rows),
    'precio_unitario': np.round(np.random.uniform(10, 500, n_rows), 2),
})

df['venta_total'] = df['cantidad'] * df['precio_unitario']
df['mes'] = df['fecha'].dt.month
df['dia_semana'] = df['fecha'].dt.day_name()

print(f"Dataset creado: {len(df)} filas")
df.head()


Dataset creado: 1000 filas


,fecha,categoria,region,vendedor,cantidad,precio_unitario,venta_total,mes,dia_semana
0,2024-04-12,Hogar,Centro,Carlos López,9,93.76,843.84,4,Friday
1,2024-12-14,Hogar,Norte,Ana García,12,214.89,2578.68,12,Saturday
2,2024-09-27,Electrónica,Norte,Ana García,19,381.07,7240.33,9,Friday
3,2024-04-16,Ropa,Norte,Laura Sánchez,3,450.00,1350.00,4,Tuesday
4,2024-03-12,Ropa,Norte,Laura Sánchez,15,51.22,768.30,3,Tuesday


In [3]:
# Guardar dataset para usar en Streamlit
df.to_csv('ventas_2024.csv', index=False)
print("✅ Dataset guardado como 'ventas_2024.csv'")


✅ Dataset guardado como 'ventas_2024.csv'


## 3. Exploración de datos y visualizaciones


In [4]:
# Métricas principales (KPIs)
total_ventas = df['venta_total'].sum()
promedio_venta = df['venta_total'].mean()
total_transacciones = len(df)
ticket_promedio = total_ventas / total_transacciones

print("📊 MÉTRICAS PRINCIPALES")
print("=" * 40)
print(f"Total Ventas: ${total_ventas:,.2f}")
print(f"Promedio por Venta: ${promedio_venta:,.2f}")
print(f"Total Transacciones: {total_transacciones:,}")
print(f"Ticket Promedio: ${ticket_promedio:,.2f}")


📊 MÉTRICAS PRINCIPALES
Total Ventas: $2,577,056.20
Promedio por Venta: $2,577.06
Total Transacciones: 1,000
Ticket Promedio: $2,577.06


In [5]:
# Ventas por categoría
ventas_categoria = df.groupby('categoria')['venta_total'].sum().sort_values(ascending=False)

fig = px.bar(
    x=ventas_categoria.index,
    y=ventas_categoria.values,
    title='Ventas por Categoría',
    labels={'x': 'Categoría', 'y': 'Ventas ($)'},
    color=ventas_categoria.values,
    color_continuous_scale='viridis'
)
fig.show()


In [6]:
# Ventas por región (pie chart)
ventas_region = df.groupby('region')['venta_total'].sum()

fig = px.pie(
    values=ventas_region.values,
    names=ventas_region.index,
    title='Distribución de Ventas por Región',
    hole=0.4
)
fig.show()


## 4. Código de la aplicación Streamlit

El siguiente código crea el archivo `app.py` que se ejecuta con `streamlit run app.py`


In [7]:
# Crear archivo app.py para Streamlit
streamlit_code = '''import streamlit as st
import pandas as pd
import plotly.express as px

# Configuración de página
st.set_page_config(page_title="Dashboard de Ventas 2024", page_icon="📊", layout="wide")

st.title("📊 Dashboard de Ventas 2024")
st.markdown("---")

# Cargar datos
@st.cache_data
def load_data():
    df = pd.read_csv("ventas_2024.csv")
    df['fecha'] = pd.to_datetime(df['fecha'])
    return df

df = load_data()

# SIDEBAR: FILTROS
st.sidebar.header("🔍 Filtros")
categorias = st.sidebar.multiselect("Categorías", options=df['categoria'].unique(), default=df['categoria'].unique())
regiones = st.sidebar.multiselect("Regiones", options=df['region'].unique(), default=df['region'].unique())

# Aplicar filtros
df_filtered = df[(df['categoria'].isin(categorias)) & (df['region'].isin(regiones))]

# MÉTRICAS
col1, col2, col3, col4 = st.columns(4)
col1.metric("Total Ventas", f"${df_filtered['venta_total'].sum():,.0f}")
col2.metric("Transacciones", f"{len(df_filtered):,}")
col3.metric("Ticket Promedio", f"${df_filtered['venta_total'].mean():,.2f}")
col4.metric("Unidades", f"{df_filtered['cantidad'].sum():,}")

st.markdown("---")

# GRÁFICOS
col_left, col_right = st.columns(2)

with col_left:
    st.subheader("🏷️ Ventas por Categoría")
    ventas_cat = df_filtered.groupby('categoria')['venta_total'].sum().sort_values(ascending=True)
    fig_cat = px.bar(x=ventas_cat.values, y=ventas_cat.index, orientation='h', color=ventas_cat.values)
    fig_cat.update_layout(showlegend=False, coloraxis_showscale=False)
    st.plotly_chart(fig_cat, use_container_width=True)

with col_right:
    st.subheader("🗺️ Ventas por Región")
    ventas_reg = df_filtered.groupby('region')['venta_total'].sum()
    fig_reg = px.pie(values=ventas_reg.values, names=ventas_reg.index, hole=0.4)
    st.plotly_chart(fig_reg, use_container_width=True)

# Línea temporal
st.subheader("📅 Evolución de Ventas Mensuales")
ventas_mes = df_filtered.groupby('mes')['venta_total'].sum().reset_index()
fig_linea = px.line(ventas_mes, x='mes', y='venta_total', markers=True)
st.plotly_chart(fig_linea, use_container_width=True)

st.markdown("---")
st.markdown("Dashboard creado con Streamlit | Ingeniería de Datos 2024")
'''

with open('app.py', 'w', encoding='utf-8') as f:
    f.write(streamlit_code)

print("✅ Archivo 'app.py' creado")
print("\\n📌 Para ejecutar: streamlit run app.py")


✅ Archivo 'app.py' creado
\n📌 Para ejecutar: streamlit run app.py
